# 計算ノードの削除

このnotebookでは、OCS OpenHPCテンプレートで作成されたSlurmクラスタから計算ノードを削除する手順について説明します。

## 前提

* 正常にジョブ実行ができる状態となっている状態を前提とします。構築途中であったり不具合のある状態の環境は対象としません。
* クラスタの管理を単純化するため、削除対象となる計算ノードは所定の方式によりIPアドレスから機械的に選択します。
* 「811-ノード数のスケジュール設定.ipynb」でスケジュール設定を行なっている場合は、このnotebookで計算ノードを削除することはできません。

## 準備

### UnitGroup名の指定

構築環境の UnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために group_vars ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### group_varsの読み込み

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、UnitGroup名に指定した 値に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py

gvars = load_group_vars(ugroup_name)

group_varsの内容を確認しておきます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

指定されたUnitGroupでスケジュール設定が行われていないことを確認します。

In [ ]:
if gvars.get("vcnode_schedule") is not None:
    raise RuntimeError("スケジュール設定が行われています")

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

## 削除対象のノード抽出

現在の計算ノード数を確認します。

In [ ]:
gvars["compute_nodes"]

削除するノード数を、`numnodes_remove`に設定します。この値は、group_varsのcompute_nodesより小さい値である必要があります。

In [ ]:
# (例)
# numnodes_remove = 1

numnodes_remove = 1

削除する計算ノード数が適切かどうかチェックします。

In [ ]:
if not isinstance(numnodes_remove, int):
    msg = f'整数を指定してください: {numnodes_remove}'
    raise RuntimeError(msg)
if numnodes_remove < 1:
    msg = f'1以上の値を指定してください: {numnodes_remove}'
    raise RuntimeError(msg)
if numnodes_remove >= gvars['compute_nodes']:
    raise RuntimeError('Error: numnodes_remove too large.')

削除する計算ノードを抽出します。

In [ ]:
hosts = gvars["compute_etc_hosts"]
current_nodes = gvars["compute_nodes"]
compute_nodes_new = gvars["compute_nodes"] - numnodes_remove
addrs_to_remove =  [x for x in hosts][compute_nodes_new:current_nodes]
hosts_to_remove = [x for x in hosts.values()][compute_nodes_new:current_nodes]
addrs_to_remove, hosts_to_remove

## ジョブスケジューリングの抑止

削除対象のノードにジョブがスケジュールされない状態に遷移させ、ジョブが割り当てられていない状態になるまで待ち合わせます。

削除対象の計算ノードをDRAIN状態に遷移させ、これ以上ジョブがスケジュールされないようにします。

In [ ]:
!ansible {ugroup_name}_master -b -a \
    "scontrol update NodeName={','.join(hosts_to_remove)} \
        State=DRAIN Reason='Node remboval'"

計算ノードの状態がDRAINEDになり、ジョブがスケジュールされない状態になるのを待ち合わせます。

In [ ]:
from time import sleep

for i in range(60):
    out = !ansible {ugroup_name}_master -b -a \
            "sinfo --noheader -o %T -N -n {','.join(hosts_to_remove)}" \
                | tail -n +2 | grep -v drained | wc -l
    if int(out[0]) == 0:
        break
    sleep(10)
else:
    raise RuntimeError('Error: timeout waiting for nodes drained.')
    
!ansible {ugroup_name}_master -a 'sinfo'

## Slurmクラスタの再構成

削除対象のノードをSlurmクラスタから削除します。

In [ ]:
!ansible {ugroup_name}_master -b -a 'scontrol delete NodeName={",".join(hosts_to_remove)}'

削除後の状態を確認します。

In [ ]:
!ansible {ugroup_name}_master -a 'sinfo'

## 削除対象のノードの終了処理

### VCノードの削除

計算ノードのUnitから、削除対象のVCノードを削除します。

計算ノードのUnitを取得します。

In [ ]:
ug = vcp.get_ugroup(gvars['ugroup_name'])
unit = ug.get_unit('compute')

現状のUnitの状態を確認します。

In [ ]:
unit.df_nodes()

対象のVCノードを削除します。

In [ ]:
for addr in addrs_to_remove:
    unit.delete_nodes(ip_address=addr)

削除後のUnitの状態を表示し、削除対象のノードが削除されていることを確認します。

In [ ]:
unit.df_nodes()

### mdx VMの削除

mdx VMを削除します。mdx以外のクラウドを利用している場合はこの節をスキップしてください。

mdxの操作のため、mdx REST API認証トークンを入力します。

In [ ]:
from getpass import getpass
mdx_token = getpass("mdx API token")

mdx REST APIエンドポイントにIPv6で接続しようとすると到達不可となる場合があるため、以下のセルを実行してIPv4での接続を強制します。

In [ ]:
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

VCP SDK mdx用プラグインモジュールを読み込みます。

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)
mdx.set_current_project_by_name(gvars['mdx_project_name'])

mdx VMを削除するためにはまず停止状態にする必要があるため、VMを強制停止します。削除するVMは後で使うことはなく、また、停止にかかる時間が短い方が望ましいため、OSのシャットダウンを経由せず強制停止します。

In [ ]:
from vcpsdk.plugins.mdx_ext import SLEEP_COUNT
from vcpsdk.plugins.mdx_ext import SLEEP_TIME_SEC
from time import sleep

for vm in hosts_to_remove:
    mdx.power_off_vm(vm, wait_for=False)
for vm in hosts_to_remove:
    for i in range(SLEEP_COUNT):
        info = mdx.get_vm_info(vm)
        if info['status'] == 'PowerOFF':
            break
        sleep(SLEEP_TIME_SEC)
    else:
        raise RuntimeError(f'Error: VM {vm} not powered off.')

VMを削除します。

In [ ]:
for vm in hosts_to_remove:
    mdx.destroy_vm(vm, wait_for=False)
for vm in hosts_to_remove:
    for i in range(SLEEP_COUNT):
        info = mdx.get_vm_info(vm)
        if info is None:
            break
        sleep(SLEEP_TIME_SEC)
    else:
        raise RuntimeError(f'Error: VM {vm} not destroyed off.')

## notebook環境側のデータ更新

### ansibleの設定更新

削除したVCノードをansibleでの操作対象から外すため、ansibleの設定を更新します。

SSHのknown_hostsファイルから、削除したノードのエントリを削除します。

In [ ]:
for addr in addrs_to_remove:
    !ssh-keygen -R {addr}

ansibleの`inventory.yml`を更新します。

In [ ]:
import yaml

inventory = {
    'all': {
        'children': {
            ug.name: {
                'children': dict([
                    (
                        f'{ug.name}_{unit.name}',
                        {
                            'hosts': dict([
                                (ip, {})
                                for ip in unit.find_ip_addresses()
                            ])
                        }
                    )
                    for unit in ug.find_units()]),
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

### group_varsの更新

ノードを削除した際に更新する必要がある、以下のパラメータを更新します。

* `compute_nodes`


In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    compute_nodes=compute_nodes_new,
)

group_varsの内容を確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml